In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Basics Cover the Most Master File"  # file without extention
sheets = "85 Sentence"  # 2 gram target, 2 gram hybrid
file_ext = "85 Sentence"
first_column = 2  # start column location number
end_column = 3

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

In [4]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [5]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

In [ ]:
df_file = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name=f"{sheets}")
#df_file.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=True)
df_file.reset_index(drop=True, inplace=True)
df_file

In [ ]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])  # "search_string" column
df_check = df_file_count[df_file_count["word_count"] > 1]
check_list = df_check["word"].to_list()
remove_list = []
df_file_var = df_file
for var_sent in df_file["search_string"]:
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    if all(result_list):
        df_file_var = df_file_var[df_file_var["search_string"] != f"{var_sent}"]
        df_file_var_count = word_count_result(df_file_var, df_file_var.columns[0:1])
        df_check = df_file_var_count[df_file_var_count["word_count"] > 1]
        check_list = df_check["word"].to_list()
        remove_list.append(var_sent)
    else:
        pass

df_file_var.reset_index(drop=True, inplace=True)
df_file_var

In [ ]:
remove_list

In [ ]:
df_result_word_count = word_count_result(df_file_var, df_file.columns[0:1])
df_result_word_count

In [ ]:
df_file_var.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample_Selected.xlsx", index=False)

In [ ]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])
df_file_count.sort_values(by="word_count", ascending=False)
df_file_count

In [ ]:
df_check = df_file_count[df_file_count["word_count"] > 2]
df_check

In [ ]:
check_list = df_check["word"].to_list()
check_list

In [ ]:
len(check_list)

In [ ]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])  # "search_string" column
df_check = df_file_count[df_file_count["word_count"] > 2]
check_list = df_check["word"].to_list()
remove_list = []
for var_sent in df_file["search_string"]:
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    if all(result_list):
        df_file = df_file[df_file["search_string"] != f"{var_sent}"]
        df_file_count = word_count_result(df_file, df_file.columns[0:1])
        df_check = df_file_count[df_file_count["word_count"] > 2]
        check_list = df_check["word"].to_list()
        remove_list.append(var_sent)
    else:
        df_file = df_file

In [ ]:
df_file

In [ ]:
remove_list

In [ ]:
word_count_result(df_file, df_file.columns[0:1])

In [ ]:
test = ["aet","önce"]

In [ ]:
test in check_list

In [ ]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])  # "search_string" column
df_check = df_file_count[df_file_count["word_count"] > 2]
check_list = df_check["word"].to_list()
i = 0
while i < len(df_file):
    var_sent in df_file.loc[i,"search_string"]
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    if all(result_list):
        df_file = df_file[df_file["search_string"] != f"{var_sent}"]
        df_file_count = word_count_result(df_file, df_file.columns[0:1])
        df_check = df_file_count[df_file_count["word_count"] > 1]
        check_list = df_check["word"].to_list()
    else:
        df_file = df_file
    i += 1

In [ ]:
df_file

In [ ]:
check_list

In [ ]:
len(check_list)

In [ ]:
while var_sent in df_file["search_string"]:
    print(var_sent)

In [ ]:
all_list = []
for var_sent in df_file["search_string"]:
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    all_list.append(result_list)

In [ ]:
all_list

In [ ]:
test_list = []
for j in all_list:
    if all(j):
        test_list.append(j)
    else:
        pass

In [ ]:
test_list

In [ ]:
result_list

In [ ]:
all(result_list)

In [ ]:
a = []

In [ ]:
a.append(True)

In [ ]:
a.append(False)

In [ ]:
a